# Tests and Errors

Many errors will arise when you develop your Python package. 
At a mature stage of development the code should be error-free and robust. This means that 
anyone should expect to be able to use it without encountering errors.
To ensure that as you continue developing your package you are not breaking some parts, leading to some errors without you 
noticing, the best way is to write a test suite.


The test suite is a set of tests that should be run automatically to check every functionality of your package every time you update its distribution.


Before going into the test part, let us recap on the different types of errors you will generally encounter.


## Types of errors in Python

In Python, there are several common **built-in exceptions** that you'll frequently encounter and might want to test against. Here are some of the main ones:

1. `ZeroDivisionError`: Raised when attempting to divide by zero.

In [10]:
result = 10 / 0  # Raises ZeroDivisionError

ZeroDivisionError: division by zero


2. `TypeError`: Raised when an operation or function is applied to an object of inappropriate type. For example, trying to add a string to an integer or passing a non-iterable to a function that expects an iterable.

In [9]:
result = 'text' + 10  # Raises TypeError

TypeError: can only concatenate str (not "int") to str


3. `ValueError`: Raised when a function receives an argument of the correct type but inappropriate value. This could happen, for instance, when trying to convert a non-numeric string to an integer.

In [8]:
number = int("abc")  # Raises ValueError

ValueError: invalid literal for int() with base 10: 'abc'



4. `IndexError`: Raised when an index is out of the range of a list, tuple, or other indexable collections. 

In [7]:
lst = [1, 2, 3]
print(lst[5])  # Raises IndexError

IndexError: list index out of range


5. `KeyError`: Raised when trying to access a dictionary with a key that doesn’t exist. This is useful for handling cases where a function requires specific dictionary keys.

In [6]:
my_dict = {"a": 1}
print(my_dict["b"])  # Raises KeyError

KeyError: 'b'


6. `AttributeError`: Raised when an invalid attribute is referenced, typically due to accessing an attribute or method that doesn’t exist in an object.

In [5]:
class MyClass:
    pass

obj = MyClass()
obj.some_method()  # Raises AttributeError

AttributeError: 'MyClass' object has no attribute 'some_method'


7. `FileNotFoundError`: Raised when trying to open a file that does not exist. It’s often used in data science to handle cases where file paths are incorrect or files are missing.


In [4]:
with open("non_existent_file.txt") as f:
    content = f.read()  # Raises FileNotFoundError

FileNotFoundError: [Errno 2] No such file or directory: 'non_existent_file.txt'


8. `OverflowError`: Raised when a numerical calculation exceeds the maximum limit for a numeric type. This is common in scientific computations where very large numbers are generated.

In [2]:
import math
result = math.exp(1000)  # Raises OverflowError on some systems


OverflowError: math range error


9. `AssertionError`: Raised when an `assert` statement fails. Useful in testing when specific conditions should be met.

In [1]:
assert 2 + 2 == 5  # Raises AssertionError

AssertionError: 



10. `RuntimeError`: A generic error raised when an error occurs that doesn’t fall into other categories. It’s often used in more complex scenarios where exceptions need custom handling.




### Exception handling

These exception allow us to use a very useful feature of Python which is called **exception handling**.

An example is more useful than words:

In [11]:
def divide(a, b):
    try:
        result = a / b
    except ZeroDivisionError:
        print("Error: Cannot divide by zero!")
        return None
    else:
        print("Division successful!")
        return result
    finally:
        print("Execution complete.")

# Example usage
print(divide(10, 2))  # Should print "Division successful!" and the result 5.0
print(divide(10, 0))  # Should print "Error: Cannot divide by zero!" and return None


Division successful!
Execution complete.
5.0
Error: Cannot divide by zero!
Execution complete.
None


Without exception handling, the program would crash. This feature allows you to handle errors gracefully and continue the execution of the program, which can mean simply exiting it but in a smooth manner, and providing a message to the user on what is going wrong.